# Disk Fragmenter

In [90]:
from collections import Counter

In [6]:
def parse_input(file_path="disk_fragmenter.txt", mode="r"):
	with open(file_path, mode) as file:
		return list(map(int, file.read().strip()))

In [154]:
def format_data(data):
	disk = []
	for i, value in enumerate(data):
		if i % 2 == 0:
			disk.extend([i//2]*value)
		else:
			disk.extend(["."]*value)
	return disk

## Part 1

In [178]:
def defragment_disk(disk):
	dot_count = Counter(disk)["."]
	defrag_disk = []
	counter = 0
	idx = len(disk) - 1

	for value in disk:
		
		if len(defrag_disk) == len(disk)-dot_count:
			break

		if value != ".":
			defrag_disk.append(value)

		else:
			while disk[idx - counter] == ".":
				counter += 1
			defrag_disk.append(disk[idx - counter])		
			counter += 1

	return defrag_disk

In [187]:
def check_sum(defragged_disk):
	suma = 0
	for i, value in enumerate(defragged_disk):
		if value != ".":
			suma += i*value 
	return suma

In [188]:
def part1(file_path):
	data = parse_input(file_path)
	disk = format_data(data)
	defragged_disk= defragment_disk(disk)
	suma = check_sum(defragged_disk)

	return suma, defragged_disk

In [189]:
suma, deffrag_disk = part1(file_path="sample.txt")
suma

1928

In [190]:
suma, deffrag_disk = part1(file_path="disk_fragmenter.txt")
suma

6471961544878

## Part 2

In [235]:
sample = parse_input(file_path="sample.txt")
sample_data = format_data(sample)

In [252]:
def find_block_from_right(data, value):
	block_size = 0
	item = value
	for i in range(len(data)-1, -1, -1):
		if data[i] == item:
			found_idx = i
			block_size += 1
		
	return found_idx, block_size

def find_sublist_from_left(big_list, small_list):
	for i in range(len(big_list)):
		if big_list[i:i+len(small_list)] == small_list:
			return i
		
	return None

def remove_block(data, block_idx, block_size, replace_with="."):
	for i in range(block_size):
		data[block_idx+i] = replace_with
	return data

In [303]:
def find_all_empty_spaces(data, empty_char="."):
	# empty_spaces = [] structure: [(start_idx, end_idx), ...]
	empty_spaces_idx = []
	empty_block_size = []
	start_idx = None
	for i, value in enumerate(data):
		if value == empty_char and start_idx is None:
			start_idx = i
		elif value != empty_char and start_idx is not None:
			empty_spaces_idx.append((start_idx, i))
			empty_block_size.append(i-start_idx)
			start_idx = None
	
	if start_idx is not None:
		empty_spaces_idx.append((start_idx, len(data)))
		empty_block_size.append(len(data)-start_idx)

	return empty_block_size, empty_spaces_idx

In [304]:
test_data = format_data(parse_input(file_path="sample.txt"))
"".join(map(str, test_data))

'00...111...2...333.44.5555.6666.777.888899'

In [309]:
empty_block_size, empty_interval_idxs = find_all_empty_spaces(data)

list(enumerate(zip(empty_block_size, empty_interval_idxs)))

[(0, (3, (2, 5))),
 (1, (3, (8, 11))),
 (2, (3, (12, 15))),
 (3, (1, (18, 19))),
 (4, (1, (21, 22))),
 (5, (1, (26, 27))),
 (6, (1, (31, 32))),
 (7, (1, (35, 36)))]

In [316]:
def defragment_by_block_improved(data):
	empty_block_size, empty_interval_idxs = find_all_empty_spaces(data)

	seen_values = set()
	for i in range(len(data)-1, -1, -1):
		current_value = data[i]
		if current_value != "." and current_value not in seen_values:
			seen_values.add(current_value)
			block_idx, block_size = find_block_from_right(data, current_value)
			for j, (empty_space, (start, end)) in enumerate(zip(empty_block_size, empty_interval_idxs)):
				if start > block_idx:
					break
				if empty_space >= block_size:
					data = remove_block(data, block_idx, block_size)
					data[start:start+block_size] = [current_value]*block_size
					empty_block_size[j] -= block_size
					empty_interval_idxs[j] = (start+block_size, end)
					break

	return data

In [317]:
test_data = format_data(parse_input(file_path="sample.txt"))
"".join(map(str, test_data))

'00...111...2...333.44.5555.6666.777.888899'

In [318]:
"".join(map(str, defragment_by_block_improved(test_data)))

'00992111777.44.333....5555.6666.....8888..'

In [270]:
def defragment_by_block(data):
	done_values = set()
	for i in range(len(data)-1, -1, -1):
		if data[i] != "." and data[i] not in done_values:
			done_values.add(data[i])
			block_idx, block_size = find_block_from_right(data, data[i])
			dots_idx = find_sublist_from_left(data[:block_idx], ["."]*block_size)
			if dots_idx is None:
				continue
			if dots_idx < block_idx:
				data[dots_idx:dots_idx+block_size] = data[block_idx:block_idx+block_size]
				data = remove_block(data, block_idx, block_size)

	return data

In [271]:
def part2(file_path):
	data = format_data(parse_input(file_path))
	data = defragment_by_block(data)
	suma = check_sum(data)

	return suma, data

In [319]:
def part2_improved(file_path):
	data = format_data(parse_input(file_path))
	data = defragment_by_block_improved(data)
	suma = check_sum(data)

	return suma, data

In [272]:
suma, _ = part2(file_path="disk_fragmenter.txt")
suma

6511178035564

In [320]:
suma, _ = part2_improved(file_path="disk_fragmenter.txt")
suma

6511178035564